# 2-way split (baseline)

In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
import random
from sklearn.naive_bayes import GaussianNB
from scipy.optimize import minimize
from kalman_filter.kalman_filter import (
    ConstantVelocityKalmanFilter, FinancialModelKalmanFilter, optimize_kalman_hyperparameters
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pywt  # Ensure you have pywavelets installed for wavelet transforms
# from sklearn.metrics import mean_squared_error, accuracy_score
# from sklearn.model_selection import ParameterGrid
# from joblib import Parallel, delayed

In [ ]:
# -----------------
# Hyperparameter Configurations
# -----------------

RANDOM_STATE = 42
WINDOW_SIZE = 10

LASSO_PARAM_GRID = {"logisticregression__C": np.logspace(-3, 2, 10)}
RF_PARAM_GRID = {"n_estimators": [50, 100, 200], "max_depth": [None, 10, 20]}
XGB_PARAM_GRID = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0]
}
NN_PARAM_GRID = {
    "hidden_size": [32, 64, 128],
    "learning_rate": [0.001, 0.01],
    "num_epochs": [50, 100]
}
LSTM_PARAM_GRID = {
    "hidden_size": [32, 64, 128],
    "num_layers": [1, 2],
    "learning_rate": [0.001, 0.01],
    "num_epochs": [50, 100]
}

# Kalman Filter Hyperparameters
CVKF_PARAM_GRID = [
    {"initial_state": np.array([0.0]), "Q_diag": [q], "R_diag": [r]}
    for q in [0.01, 0.1, 1.0, 10.0]
    for r in [0.01, 0.1, 1.0, 10.0]
]
FMKF_PARAM_GRID = [
    {"initial_state": np.array([0.0]), "Q_diag": [q], "R_diag": [r], "alpha": [a], "beta": [b]}
    for q in [0.01, 0.1, 1.0, 10.0]
    for r in [0.01, 0.1, 1.0, 10.0]
    for a in [0.4, 0.6, 0.8, 1.0]
    for b in [0.05, 0.1, 0.2, 0.4]
]


# -----------------
# Utility Functions
# -----------------
def two_way_split(X, y, train_size=0.6, test_size=0.4):
    """Split data into training and testing subsets chronologically."""
    total_len = len(X)

    train_len = round(total_len * train_size)
    test_len = total_len - train_len

    train_idx = range(0, train_len)
    test_idx = range(train_len, total_len)

    return (
        X.iloc[train_idx], X.iloc[test_idx],
        y.iloc[train_idx], y.iloc[test_idx]
    )

# def five_way_split(X, y, train_size=0.5, val1_size=0.15, val2_size=0.1, kalman_size=0.1, test_size=0.15):
    # """Split data into five subsets."""
    # total_len = len(X)

    # train_len = round(total_len * train_size)
    # val1_len = round(total_len * val1_size)
    # val2_len = round(total_len * val2_size)
    # kalman_len = round(total_len * kalman_size)
    # test_len = total_len - train_len - val1_len - val2_len - kalman_len

    # train_idx = range(0, train_len)
    # val1_idx = range(train_len, train_len + val1_len)
    # val2_idx = range(train_len + val1_len, train_len + val1_len + val2_len)
    # kalman_idx = range(train_len + val1_len + val2_len, train_len + val1_len + val2_len + kalman_len)
    # test_idx = range(train_len + val1_len + val2_len + kalman_len, total_len)

    # return (
    #     X.iloc[train_idx], X.iloc[val1_idx], X.iloc[val2_idx], X.iloc[kalman_idx], X.iloc[test_idx],
    #     y.iloc[train_idx], y.iloc[val1_idx], y.iloc[val2_idx], y.iloc[kalman_idx], y.iloc[test_idx]
    # )


def optimize_model_hyperparameters(model_fn, param_grid, X_train, y_train, validation_data, n_jobs=1):
    """
    Performs hyperparameter optimization using GridSearchCV.

    Args:
        model_fn: A callable that returns an instance of the model.
        param_grid: Dictionary of hyperparameters to search.
        X_train: Training features.
        y_train: Training labels.
        validation_data: Tuple (X_val, y_val) for validation.
        n_jobs: Number of parallel jobs for GridSearchCV.

    Returns:
        best_model: The best model after GridSearchCV.
        best_params: The best parameters from the search.
    """
    model = model_fn()
    grid_search = GridSearchCV(
        model,
        param_grid,
        scoring='roc_auc',
        cv=5,
        n_jobs=n_jobs,
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_params_


def calculate_classification_metrics(y_true, y_pred, y_pred_proba=None):
    """
    Calculate classification metrics including Accuracy, Precision, Recall, F1, and AUC.

    Args:
        y_true (array-like): True labels.
        y_pred (array-like): Predicted labels.
        y_pred_proba (array-like, optional): Predicted probabilities for the positive class.

    Returns:
        dict: Dictionary of calculated metrics.
    """
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0)
    }

    if y_pred_proba is not None:
        metrics["AUC"] = roc_auc_score(y_true, y_pred_proba)

    return metrics

from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed


def preprocess_data_with_advanced_features(data_frame, target_column, lag_steps=None, rolling_window=10):
    """
    Preprocess data for time series modeling with advanced feature engineering.
    Ensures no data leakage by strictly using past and current data for feature generation.

    Args:
        data_frame (str): Variable name of loaded pandas data frame.
        target_column (str): Target column name.
        lag_steps (list): List of lag steps for feature engineering.
        rolling_window (int): Window size for rolling features.

    Returns:
        tuple: Feature DataFrame (X) and target series (y).
    """
    # Load data and parse dates
    data = data_frame
    data.index = pd.to_datetime(data.index, errors='coerce')  # Ensure index is datetime
    assert data.index.is_monotonic_increasing, "Dataset is not sorted by time."

    # Fill missing values in the target column
    data[target_column] = data[target_column].interpolate(method='linear').bfill()

    # Initialize feature storage
    features = []
    indices = []

    for end_idx in range(rolling_window, len(data)):
        # Define the current window
        window = data.iloc[end_idx - rolling_window:end_idx]

        # Compute features for the current timestamp
        current_features = {}

        # Rolling statistics
        signal_cols = [col for col in data.columns if col not in ['patient', 'newtest', 'target', 'event1', 'event2', 'event3', 'event4', 'sleepstage']]  # sleepstage excluded as categorical variable
        for col in signal_cols:
            current_features[f'{col}_roll_mean'] = window[col].mean()
            current_features[f'{col}_roll_std'] = window[col].std()

        # Lagged features
        if lag_steps:
            for lag in lag_steps:
                if end_idx - lag >= 0:
                    current_features[f'{target_column}_lag{lag}'] = data[target_column].iloc[end_idx - lag]

        # Fourier Transform Features
        for col in signal_cols:
            fourier_transform = np.abs(np.fft.fft(window[col].fillna(0)))
            current_features[f'{col}_fft_max'] = np.max(fourier_transform)
            current_features[f'{col}_fft_mean'] = np.mean(fourier_transform)

        # Wavelet Transform Features
        for col in signal_cols:
            coeffs = pywt.wavedec(window[col].fillna(0), 'db1', level=3)
            current_features[f'{col}_wavelet_approx'] = coeffs[0].mean()
            current_features[f'{col}_wavelet_detail1'] = coeffs[1].mean()
            current_features[f'{col}_wavelet_detail2'] = coeffs[2].mean()

        # Add features and corresponding index
        features.append(current_features)
        indices.append(data.index[end_idx])

    # Convert features to DataFrame
    feature_df = pd.DataFrame(features, index=indices)

    # Align target values
    y = data.loc[feature_df.index, target_column]

    return feature_df, y

In [3]:
# -----------------
# Load and Preprocess Data
# -----------------

def load_and_preprocess_data(dataframe):
    # Load and preprocess data with advanced features
    X, y = preprocess_data_with_advanced_features(
        data_frame=dataframe,
        target_column='target',
        lag_steps=[1, 2, 3],
        rolling_window=10
    )

    # Perform five-way split
    X_train, X_test, y_train, y_test = two_way_split(
        X, y, train_size=0.6, test_size=0.4
    )
    
    return X_train, X_test, y_train, y_test

In [4]:
# Load data
master_df = pd.read_stata('../data/stmary/processed-data/combined-patient-data-1_00.dta')

In [5]:
# Group master dataframe by 'patient' and 'newtest' pairs (i.e. by each unique patient data)
# Access or initialize each dataframe like: group_dict[('pid100100', 0)]
group_dict = {
    (val1, val2): data
    for (val1, val2), data in master_df.groupby(['patient', 'newtest'])
}

In [6]:
# Preprocess dataframes
for group_key, subset_df in group_dict.items():
    subset_df['target'] = subset_df[['event1', 'event2', 'event3', 'event4']].apply(lambda x: 1 if 'Hypopnea' in x.values or 'Apnea Obstructive' in x.values or 'Apnea Central' in x.values or 'Apnea Mixed' in x.values else 0, axis=1)

    cols = list(subset_df.columns)
    cols.remove('target')
    cols.insert(3, 'target')
    subset_df = subset_df[cols]

    subset_df.set_index('timess', inplace=True)
    
    group_dict[group_key] = subset_df

In [7]:
Xy_before_dropnan = load_and_preprocess_data(group_dict[('pid100816', 0)])

In [8]:
# Dropping NaN from Xs and corresponding rows from ys
def drop_nan_from_Xy(X_train, X_test, y_train, y_test):
    # Drop rows with NaN from X
    X_train_cleaned = X_train.dropna()
    X_test_cleaned = X_test.dropna()
    
    # Drop corresponding rows from y
    y_train_cleaned = y_train[X_train_cleaned.index]
    y_test_cleaned = y_test[X_test_cleaned.index]
    
    return (
        X_train_cleaned, X_test_cleaned,
        y_train_cleaned, y_test_cleaned
    )

In [9]:
X_train, X_test, y_train, y_test = drop_nan_from_Xy(*Xy_before_dropnan)

In [10]:
# -----------------
# Logistic Regression
# -----------------
log_reg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logisticregression', LogisticRegression(class_weight='balanced', max_iter=2000))  # Handle class imbalance
])
log_reg_grid = GridSearchCV(log_reg_pipeline, LASSO_PARAM_GRID, cv=5, scoring='roc_auc')
log_reg_grid.fit(X_train, y_train)
log_reg_model = log_reg_grid.best_estimator_

log_reg_preds = log_reg_model.predict(X_test)
log_reg_metrics = {
    "Accuracy": accuracy_score(y_test, log_reg_preds),
    "Precision": precision_score(y_test, log_reg_preds, zero_division=0),  # Avoid warning
    "Recall": recall_score(y_test, log_reg_preds, zero_division=0),
    "F1": f1_score(y_test, log_reg_preds, zero_division=0),
    "AUC": roc_auc_score(y_test, log_reg_model.predict_proba(X_test)[:, 1])
}
print("Logistic Regression Metrics:", log_reg_metrics)

Logistic Regression Metrics: {'Accuracy': 0.9997589324149806, 'Precision': 0.9961285327138986, 'Recall': 0.9930528753377075, 'F1': 0.9945883262466177, 'AUC': np.float64(0.9978413416551788)}


# 2-way split (only transitioning 1s)

In [11]:
df_copy = group_dict[('pid100816', 0)].copy()

# Create a lagged target variable
df_copy['target_lag1'] = df_copy['target'].shift(1)

# Filter the dataset to only observations where previous state of target was 0
df_transition = df_copy[df_copy['target_lag1'] == 0]
df_transition.drop('target_lag1', axis=1, inplace=True)

/tmp/ipykernel_1126788/4138185884.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transition.drop('target_lag1', axis=1, inplace=True)


In [12]:
Xy_before_dropnan = load_and_preprocess_data(df_transition)

/tmp/ipykernel_1126788/2857536397.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[target_column] = data[target_column].interpolate(method='linear').bfill()


In [14]:
X_train, X_test, y_train, y_test = drop_nan_from_Xy(*Xy_before_dropnan)

In [15]:
# -----------------
# Logistic Regression
# -----------------
log_reg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logisticregression', LogisticRegression(class_weight='balanced', max_iter=2000))  # Handle class imbalance
])
log_reg_grid = GridSearchCV(log_reg_pipeline, LASSO_PARAM_GRID, cv=5, scoring='roc_auc')
log_reg_grid.fit(X_train, y_train)
log_reg_model = log_reg_grid.best_estimator_

log_reg_preds = log_reg_model.predict(X_test)
log_reg_metrics = {
    "Accuracy": accuracy_score(y_test, log_reg_preds),
    "Precision": precision_score(y_test, log_reg_preds, zero_division=0),  # Avoid warning
    "Recall": recall_score(y_test, log_reg_preds, zero_division=0),
    "F1": f1_score(y_test, log_reg_preds, zero_division=0),
    "AUC": roc_auc_score(y_test, log_reg_model.predict_proba(X_test)[:, 1])
}
print("Logistic Regression Metrics:", log_reg_metrics)

Logistic Regression Metrics: {'Accuracy': 0.9594838351602608, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0, 'AUC': np.float64(0.32683705019406617)}


# 2-way split (no lagged features)

In [16]:
def preprocess_data_with_advanced_features(data_frame, target_column, lag_steps=None, rolling_window=10):
    """
    Preprocess data for time series modeling with advanced feature engineering.
    Ensures no data leakage by strictly using past and current data for feature generation.

    Args:
        data_frame (str): Variable name of loaded pandas data frame.
        target_column (str): Target column name.
        lag_steps (list): List of lag steps for feature engineering.
        rolling_window (int): Window size for rolling features.

    Returns:
        tuple: Feature DataFrame (X) and target series (y).
    """
    # Load data and parse dates
    data = data_frame
    data.index = pd.to_datetime(data.index, errors='coerce')  # Ensure index is datetime
    assert data.index.is_monotonic_increasing, "Dataset is not sorted by time."

    # Fill missing values in the target column
    data[target_column] = data[target_column].interpolate(method='linear').bfill()

    # Initialize feature storage
    features = []
    indices = []

    for end_idx in range(rolling_window, len(data)):
        # Define the current window
        window = data.iloc[end_idx - rolling_window:end_idx]

        # Compute features for the current timestamp
        current_features = {}

        # Rolling statistics
        signal_cols = [col for col in data.columns if col not in ['patient', 'newtest', 'target', 'event1', 'event2', 'event3', 'event4', 'sleepstage']]  # sleepstage excluded as categorical variable
        for col in signal_cols:
            current_features[f'{col}_roll_mean'] = window[col].mean()
            current_features[f'{col}_roll_std'] = window[col].std()

        # # Lagged features
        # if lag_steps:
        #     for lag in lag_steps:
        #         if end_idx - lag >= 0:
        #             current_features[f'{target_column}_lag{lag}'] = data[target_column].iloc[end_idx - lag]

        # Fourier Transform Features
        for col in signal_cols:
            fourier_transform = np.abs(np.fft.fft(window[col].fillna(0)))
            current_features[f'{col}_fft_max'] = np.max(fourier_transform)
            current_features[f'{col}_fft_mean'] = np.mean(fourier_transform)

        # Wavelet Transform Features
        for col in signal_cols:
            coeffs = pywt.wavedec(window[col].fillna(0), 'db1', level=3)
            current_features[f'{col}_wavelet_approx'] = coeffs[0].mean()
            current_features[f'{col}_wavelet_detail1'] = coeffs[1].mean()
            current_features[f'{col}_wavelet_detail2'] = coeffs[2].mean()

        # Add features and corresponding index
        features.append(current_features)
        indices.append(data.index[end_idx])

    # Convert features to DataFrame
    feature_df = pd.DataFrame(features, index=indices)

    # Align target values
    y = data.loc[feature_df.index, target_column]

    return feature_df, y

In [17]:
# -----------------
# Load and Preprocess Data
# -----------------

def load_and_preprocess_data(dataframe):
    # Load and preprocess data with advanced features
    X, y = preprocess_data_with_advanced_features(
        data_frame=dataframe,
        target_column='target',
        lag_steps=[1, 2, 3],
        rolling_window=10
    )

    # Perform five-way split
    X_train, X_test, y_train, y_test = two_way_split(
        X, y, train_size=0.6, test_size=0.4
    )
    
    return X_train, X_test, y_train, y_test

In [ ]:
Xy_before_dropnan = load_and_preprocess_data(group_dict[('pid100816', 0)])

In [ ]:
X_train, X_test, y_train, y_test = drop_nan_from_Xy(*Xy_before_dropnan)

In [ ]:
# -----------------
# Logistic Regression
# -----------------
log_reg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logisticregression', LogisticRegression(class_weight='balanced', max_iter=2000))  # Handle class imbalance
])
log_reg_grid = GridSearchCV(log_reg_pipeline, LASSO_PARAM_GRID, cv=5, scoring='roc_auc')
log_reg_grid.fit(X_train, y_train)
log_reg_model = log_reg_grid.best_estimator_

log_reg_preds = log_reg_model.predict(X_test)
log_reg_metrics = {
    "Accuracy": accuracy_score(y_test, log_reg_preds),
    "Precision": precision_score(y_test, log_reg_preds, zero_division=0),  # Avoid warning
    "Recall": recall_score(y_test, log_reg_preds, zero_division=0),
    "F1": f1_score(y_test, log_reg_preds, zero_division=0),
    "AUC": roc_auc_score(y_test, log_reg_model.predict_proba(X_test)[:, 1])
}
print("Logistic Regression Metrics:", log_reg_metrics)